In [1]:
# Autoreload to see the changes in the Python proj files: https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

# 0) IMPORTS

In [2]:
import pandas as pd
import numpy as np
import sys
import os

In [3]:
!pwd

/home/realmistic


In [4]:
# There is a problem with import in Jupiter notebook vs. the standard Python proj 
# https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
# https://github.com/microsoft/vscode-jupyter/issues/3605
sys.path.append('/home/realmistic/PythonInvest-basic-fin-analysis/mlops_stocks_day_trading/')

os.chdir ('//home/realmistic/PythonInvest-basic-fin-analysis/mlops_stocks_day_trading/')
!pwd

/home/realmistic/PythonInvest-basic-fin-analysis/mlops_stocks_day_trading


# 1) Test repo class (SqlLite implementation with local file)

In [6]:
import load_data_sqlite3
# load by default tickers.tb
repo = load_data_sqlite3.DataRepositorySqlLite()
repo

['tickers.db']


In [7]:
repo.fetch_ticker_incremental(ticker='GOOG', period_days=7)

[*********************100%***********************]  1 of 1 completed
downloaded records (5, 7) with min_date 2022-06-27 00:00:00 and max_date 2022-07-01 00:00:00
Successfully fetched GOOG for more 7


/home/realmistic/PythonInvest-basic-fin-analysis/mlops_stocks_day_trading/load_data_sqlite3.py:132: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tickers_df = self.tickers_df.append(incremental_df, ignore_index=True)


In [8]:
repo.tickers_df

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,GOOG,87.876999,88.032501,85.392502,86.412003,86.412003,38038000
1,2021-01-05,GOOG,86.250000,87.383499,85.900749,87.045998,87.045998,22906000
2,2021-01-06,GOOG,85.131500,87.400002,84.949997,86.764503,86.764503,52042000
3,2021-01-07,GOOG,87.002998,89.419998,86.852501,89.362503,89.362503,45300000
4,2021-01-08,GOOG,89.399002,90.491997,88.676750,90.360497,90.360497,41012000
...,...,...,...,...,...,...,...,...
372,2022-06-27,GOOG,118.934998,119.250000,116.000748,116.622498,116.622498,32840000
373,2022-06-28,GOOG,116.350998,117.856499,112.444000,112.571503,112.571503,28232000
374,2022-06-29,GOOG,112.148499,113.664497,111.554001,112.256500,112.256500,18628000
375,2022-06-30,GOOG,110.499496,111.329803,107.309998,109.372498,109.372498,38046000


In [9]:
repo.existing_db_files

['tickers.db']

# 2) Manual implementation - connect to DB, API call saving data to Pandas, flush a df to an SQLLite file

In [15]:
import sqlite3

#Connecting to sqlite db
conn = sqlite3.connect("data/tickers.db")
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [16]:
# List of all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
print(cursor.fetchall())

# check if there is an index (Pandas flush to SQLLite doesn't define a Primary Key and Index - you need to redefine it manually)
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'index';")
print(cursor.fetchall())

# Table schema 
cursor.execute("SELECT sql FROM sqlite_master WHERE name=?;",['tickers'])
rez = cursor.fetchall()[0][0]
print(rez)

# if you want to drop a table
# cursor.execute('DROP TABLE tickers')

[('tickers',)]
[('sqlite_autoindex_tickers_1',)]
CREATE TABLE "tickers" (
                "Date" TIMESTAMP,
                "Ticker" TEXT,
                "Open" REAL,
                "High" REAL,
                "Low" REAL,
                "Close" REAL,
                "Adj Close" REAL,
                "Volume" INTEGER,
                PRIMARY KEY (Date, Ticker)
                )


In [26]:
p2= pd.read_sql('select * from tickers', conn)
p2

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04 00:00:00,GOOG,87.876999,88.032501,85.392502,86.412003,86.412003,38038000
1,2021-01-05 00:00:00,GOOG,86.250000,87.383499,85.900749,87.045998,87.045998,22906000
2,2021-01-06 00:00:00,GOOG,85.131500,87.400002,84.949997,86.764503,86.764503,52042000
3,2021-01-07 00:00:00,GOOG,87.002998,89.419998,86.852501,89.362503,89.362503,45300000
4,2021-01-08 00:00:00,GOOG,89.399002,90.491997,88.676750,90.360497,90.360497,41012000
...,...,...,...,...,...,...,...,...
644,2022-01-25 00:00:00,None,128.435501,129.338501,126.377998,126.735497,126.735497,36008000
645,2022-01-26 00:00:00,None,130.592499,132.807495,127.153503,129.240005,129.240005,39630000
646,2022-01-27 00:00:00,None,131.360992,132.609955,128.945007,129.121002,129.121002,30248000
647,2022-01-28 00:00:00,None,130.000000,133.370499,128.694504,133.289505,133.289505,30518000


In [21]:
import datetime
max_data_date = datetime.datetime.strptime("2021-01-01","%Y-%m-%d")

period_days=30
start_date = max_data_date + datetime.timedelta(days=1)
end_date = max_data_date + datetime.timedelta(days=1+period_days)  

print(f'Period is {start_date} to {end_date}')

Period is 2021-01-02 00:00:00 to 2021-02-01 00:00:00


In [22]:
import yfinance as yf
ticker='GOOG'
incremental_df = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [23]:
# Define new field
incremental_df['Ticker'] = ticker

In [24]:
data = yf.download("GOOG", start="2021-01-01", end="2022-02-01")
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,87.876999,88.032501,85.392502,86.412003,86.412003,38038000
2021-01-05,86.250000,87.383499,85.900749,87.045998,87.045998,22906000
2021-01-06,85.131500,87.400002,84.949997,86.764503,86.764503,52042000
2021-01-07,87.002998,89.419998,86.852501,89.362503,89.362503,45300000
2021-01-08,89.399002,90.491997,88.676750,90.360497,90.360497,41012000
...,...,...,...,...,...,...
2022-01-25,128.435501,129.338501,126.377998,126.735497,126.735497,36008000
2022-01-26,130.592499,132.807495,127.153503,129.240005,129.240005,39630000
2022-01-27,131.360992,132.609955,128.945007,129.121002,129.121002,30248000


In [25]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
# Flush data or APPEND (the latter is better)
data.to_sql(name='tickers', 
            index=True,
            # index_label='',
            if_exists='append',
            con=conn)

272

In [27]:
conn.close()